In [ ]:
# Tell jupyter notebook to autoload config file
%load_ext autoreload
%autoreload 2

In [ ]:
import folium as fl
from folium.plugins import MarkerCluster
from config import *

## Import created dataset

In [ ]:
stops = pd.read_csv(os.path.join(gtfs_path, 'stops.txt'), dtype={"stop_id": str, "parent_station": str})
trips = pd.read_csv(os.path.join(gtfs_path, 'trips.txt'), dtype={"trip_id": str})
routes = pd.read_csv(os.path.join(gtfs_path, 'routes.txt'))
data = pd.read_csv(data_path, dtype={"from_stop": str, "to_stop": str, "trip_id": str})
print(f"Using \"{gtfs_dataset}\" dataset with {len(data.index)} rows/edges")

## Graph structure

In [ ]:
class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = []

    def add_node(self, value):
        if value not in self.nodes:
            self.nodes.add(value)

    def add_edge(self, trip_id, from_node, to_node, departure_time, arrival_time):
        self.add_node(from_node)
        self.add_node(to_node)

        edge = Edge(trip_id, from_node, to_node, departure_time, arrival_time)
        self.edges.append(edge)
        
class Edge:
    def __init__(self, trip_id, from_node, to_node, departure_time, arrival_time):
        self.trip_id = trip_id
        self.from_node = from_node
        self.to_node = to_node
        self.departure_time = departure_time
        self.arrival_time = arrival_time


def calculate_travel_time(arrival_time_at_node, edge):
    arrival_time_at_node_datetime = datetime.combine(datetime.min, arrival_time_at_node)
    departure_datetime = datetime.combine(datetime.min, edge.departure_time)
    arrival_datetime = datetime.combine(datetime.min, edge.arrival_time)
    while arrival_datetime < departure_datetime:  # Handle crossing midnight
        arrival_datetime += timedelta(days=1)
    travel_time = (arrival_datetime - departure_datetime).total_seconds()  # Travel time between two stops
    if edge.trip_id != '0' and edge.trip_id != '-1':  # Ignore wait time for walking
        travel_time += (departure_datetime - arrival_time_at_node_datetime).total_seconds()  # Wait time at the station
    return travel_time


def bellman_ford(graph, source, start_time):
    # Step 1: Initialize durations from source to all other nodes as infinity
    durations = {node: float('inf') for node in graph.nodes}
    predecessors = {node: None for node in graph.nodes}
    departure_times = {node: None for node in graph.nodes}
    arrival_times = {node: None for node in graph.nodes}
    trip_ids = {node: None for node in graph.nodes}
    # Set duration to the source itself as 0
    durations[source] = 0
    # Set arrival time at source as start time
    arrival_times[source] = start_time
    relaxation_completed = False

    print(f"Going through {len(graph.nodes)} nodes and {len(graph.edges)} edges")

    # Step 2: Relax all edges |V| - 1 times
    for _ in range(len(graph.nodes) - 1):
        updated = False
        for edge in graph.edges:
            if arrival_times[edge.from_node] is None:
                continue
            # Skip edges that have departure time before arrival time at the node
            if edge.departure_time < arrival_times[edge.from_node] and edge.trip_id != '0' and edge.trip_id != '-1':
                continue

            travel_time = calculate_travel_time(arrival_times[edge.from_node], edge)

            if durations[edge.from_node] != float('inf') and durations[edge.from_node] + travel_time < durations[edge.to_node]:
                durations[edge.to_node] = durations[edge.from_node] + travel_time
                if edge.trip_id == '0': # Changing platform
                    arrival_times[edge.to_node] = arrival_times[edge.from_node]
                    departure_times[edge.to_node] = arrival_times[edge.from_node]
                elif edge.trip_id == '-1': # Walking based on pathways.txt file
                    arrival_times[edge.to_node] = (datetime.combine(datetime.min, arrival_times[edge.from_node]) + timedelta(seconds=travel_time)).time()
                    departure_times[edge.to_node] = arrival_times[edge.from_node]
                else:
                    arrival_times[edge.to_node] = edge.arrival_time
                    departure_times[edge.to_node] = edge.departure_time
                predecessors[edge.to_node] = edge.from_node
                trip_ids[edge.to_node] = edge.trip_id
                updated = True
                # if edge.trip_id != '0' and edge.trip_id != '-1':
                #     print(f"Relaxed edge {edge.from_node} -> {edge.to_node} ({edge.trip_id}): New distance = {durations[edge.to_node]}")
        if not updated:
            relaxation_completed = True
            break
    
    # Step 3: Check for negative-weight cycles
    if not relaxation_completed:
        for edge in graph.edges:
            if arrival_times[edge.from_node] is None:
                continue
            # Skip edges that have departure time before arrival time at the node
            if edge.departure_time < arrival_times[edge.from_node] and edge.trip_id != '0' and edge.trip_id != '-1':
                continue
    
            travel_time = calculate_travel_time(arrival_times[edge.from_node], edge)
    
            if durations[edge.from_node] != float('inf') and durations[edge.from_node] + travel_time < durations[edge.to_node]:
                raise ValueError("Graph contains negative weight cycle")

    return durations, predecessors, departure_times, arrival_times, trip_ids


def get_shortest_path(graph, source, destination, start_time):
    duration, predecessors, departure_times, arrival_times, trip_ids = bellman_ford(graph, source, start_time)
    
    full_path = []
    current_node = destination

    while current_node != source:
        full_path.append(current_node)
        current_node = predecessors[current_node]

    full_path.append(source)
    full_path.reverse()

    return duration[destination], full_path, departure_times, arrival_times, trip_ids


## Building the network

In [ ]:
def convert_time(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    while hours >= 24:
        hours -= 24
    return time(hour=hours, minute=minutes, second=seconds)

In [ ]:
graph = Graph()
counter = 0
for row in data.itertuples():
    counter += 1
    if counter % 100000 == 0 or counter == len(data.index):
        print('Progress : {}%'.format(np.round(counter / len(data.index) * 100, 2)))

    departure_time = convert_time(row.departure_time)
    arrival_time = convert_time(row.arrival_time)
    graph.add_edge(row.trip_id, row.from_stop, row.to_stop, departure_time, arrival_time)
    if row.trip_id == '0':  # Bidirectional edge for changing platforms
        graph.add_edge(row.trip_id, row.to_stop, row.from_stop, departure_time, arrival_time)

## Find the shortest route

In [ ]:
parent_stations = stops[stops['location_type'] == 1]


def get_station(stop_name):
    found_station = parent_stations[parent_stations['stop_name'].str.contains(stop_name, regex=False, case=False)]
    if found_station.empty:
        found_station = stops[stops['stop_name'].str.contains(stop_name, regex=False, case=False)]
    if found_station.empty:
        print(f"No stops found with stop name {stop_name}")
    return found_station['stop_id'].astype(str).iloc[0], found_station['stop_name'].iloc[0]


def get_train_name(trip_id):
    if not trip_id or trip_id == '0':
        return ""
    if trip_id == '-1':
        return "Walking"
    route_id = trips[trips['trip_id'] == trip_id]['route_id'].iloc[0]
    train_name = routes[routes['route_id'] == route_id]['route_short_name'].iloc[0]
    return train_name


def find_shortest_route(source, destination, start_time):
    duration, path, departure_times, arrival_times, trip_ids = get_shortest_path(graph, source, destination, start_time)

    trip_id = [trip_ids[i] for i in path]
    trip_id = [f"{get_train_name(i)}" if i == '-1' or i == '0' else f"{get_train_name(i)} ({i})" if i else "" for i in trip_id]
    departure_time = ["" if trip_ids[i] == '0' else departure_times[i] for i in path]
    arrival_time = ["" if trip_ids[i] == '0' else arrival_times[i] for i in path]
    departure_time = departure_time[1:] + [""]

    route = pd.merge(
        pd.DataFrame({'stop_id' : path, 'arrival_time': arrival_time, 'departure_time': departure_time, 'trip_id': trip_id}),
        stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],
        on='stop_id'
    ).reset_index(drop=True)
    print('Travel time     : {}'.format(str(timedelta(seconds=duration))))
    print(' ')
    print(route[['stop_id', 'trip_id', 'stop_name', 'arrival_time', 'departure_time']])
    return route

In [ ]:
from_station_id, from_station_name = get_station("Berlin Haupt")
to_station_id, to_station_name = get_station("München Hbf")
start_time = "11:00:00"
print(f"Calculating shortest route from \"{from_station_name}\" ({from_station_id}) to \"{to_station_name}\" ({to_station_id}) at {start_time}")

In [ ]:
route = find_shortest_route(from_station_id, to_station_id, datetime.strptime(start_time, '%H:%M:%S').time())

## Plot the shortest route on map

In [ ]:
# Loads the map roughly in the middle of all stations
route_map = fl.Map(location=[route['stop_lat'].median(), route['stop_lon'].median()], zoom_start=8)
stop_clusters = MarkerCluster()
points = []

for row in route.itertuples():
    points.append([row.stop_lat, row.stop_lon])
    stop_clusters.add_child(fl.Marker(location=[row.stop_lat, row.stop_lon], popup=row.stop_name))

route_map.add_child(stop_clusters)
line = fl.PolyLine(points, color="red", weight=3, opacity=0.5)
route_map.add_child(line)
route_map